# CLV

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mahdi-ebrahimi-per/AI/blob/main/8-%20CLV/CLV.ipynb)




In [1]:
import numpy as np
import pandas as pd
from prettytable import PrettyTable
import matplotlib.pyplot as plt
import math

from persiantools.jdatetime import JalaliDate
import datetime
from dateutil.relativedelta import relativedelta
import operator

from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

# import seaborn as sns

from colorama import Fore

from six import StringIO #DM
from IPython.display import Image, display
import pydot


In [2]:
# Jalali to Gregorian

# index = 0
# for date in DF[DATE_COL]:
#     sploted_date = date.split("/")
#     toGreg = JalaliDate(sploted_date[0], sploted_date[1], sploted_date[2]).to_gregorian()
#     index += 1
#     DF[DATE_COL][index] = toGreg.strtime("%Y/%M/%D")
    




### Load Transaction CSV
    - اطلاعات مربوط به تراکنش های مربوط به ترمینال هاست

In [11]:
# Load Data Frame
# Trans_df = pd.read_csv("TRANSACTIONS.csv", encoding="utf-8-sig")
Trans_df = pd.read_excel("TRANSACTIONS.xlsx")

Trans_df = Trans_df.iloc[:, :9] #ignore last 2 columns
# print(Trans_df.head())


# Unique Value
unique_counts = PrettyTable()
unique_counts.field_names = ["Feature", "Unique Value"]

for column in Trans_df.columns:
    count = Trans_df[column].nunique()
    unique_counts.add_row([column, count])


print(unique_counts)    


# NaN Values
NaN_include_flag = False
Features_includes_NaN = 0
for f in list(Trans_df.isna().sum()):
    if f > 0:
        NaN_include_flag = True
        Features_includes_NaN += 1

if NaN_include_flag:
    print(Fore.RED + f"DataFrame Includes {sum(list(Trans_df.isna().sum()))} NaN Values in {Features_includes_NaN} Features 👎🏼" + Fore.RESET)

else:
    print(Fore.GREEN + "DataFrame Haven't any NaN Values 👍🏼" + Fore.RESET)


+---------------+--------------+
|    Feature    | Unique Value |
+---------------+--------------+
|  TERMINAL_NO  |      0       |
|    TRANDATE   |      0       |
|  RECENCYDATE  |      0       |
|  TRANNUMDAYS  |      0       |
|   FREQUENCY   |      0       |
|     AMOUNT    |      0       |
| TRAN_PURCHASE |      0       |
|  TRAN_CHARGE  |      0       |
|  TRAN_BALANCE |      0       |
+---------------+--------------+
DataFrame Haven't any NaN Values 👍🏼


| TERMINAL_NO | TRANDATE | RECENCYDATE | TRANNUMDAYS | FREQUENCY | AMOUNT | TC | TRAN_PURCHASE | TRAN_CHARGE | TRAN_BALANCE |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| شماره ترمینال | تاریخ | آخرین خرید | تعداد روز هایی که در ماه تراکنش داشته | تعداد تراکنش | مبلغ کل تراکنش | کارمزد | تراکنش خرید | تراکنش شارژ | تراکنش دریافت موجودی |

### Load Merchant CSV
    -  ها است PSP این دیتاست مربوط به اطلاعات  

In [4]:
# Load Data Frame
Merch_df = pd.read_excel("MERCHANTS.xlsx")

Merch_df = Merch_df.iloc[:, :23] #drop fetch


# Unique Value
unique_counts = PrettyTable()
unique_counts.field_names = ["Feature", "Unique Value"]

for column in Merch_df.columns:
    count = Merch_df[column].nunique()
    unique_counts.add_row([column, count])


print(unique_counts)    


# NaN Values
NaN_include_flag = False
Features_includes_NaN = 0
for f in list(Merch_df.isna().sum()):
    if f > 0:
        NaN_include_flag = True
        Features_includes_NaN += 1

if NaN_include_flag:
    print(Fore.RED + f"DataFrame Includes {sum(list(Trans_df.isna().sum()))} NaN Values in {Features_includes_NaN} Features 👎🏼" + Fore.RESET)

else:
    print(Fore.GREEN + "DataFrame Haven't any NaN Values 👍🏼" + Fore.RESET)


+-----------------+--------------+
|     Feature     | Unique Value |
+-----------------+--------------+
|   TERMINAL_NO   |      0       |
| MAININSTALLDATE |      0       |
|  DISABLED_DATE  |      0       |
|  CATEGORY_NAME  |      0       |
|   CATEGORYCODE  |      0       |
|   PROVINCENAME  |      0       |
|   PROVINCECODE  |      0       |
|     CITYNAME    |      0       |
|     CITYCODE    |      0       |
|   PROJECTNAME   |      0       |
|   PROJECTCODE   |      0       |
|    AGENTNAME    |      0       |
|    AGENTCODE    |      0       |
|  TERMINALSTATUS |      0       |
|     POSTYPE     |      0       |
|     POSMODEL    |      0       |
|     POSBRAND    |      0       |
| WORKFLOWCAPTION |      0       |
|       TEL1      |      0       |
|   FIRSTNAMEFA   |      0       |
|    LASTNAMEFA   |      0       |
|  OFFICEADDRESS  |      0       |
|      MOBILE     |      0       |
+-----------------+--------------+
DataFrame Haven't any NaN Values 👍🏼


### Join two DataFrame according to "Merchant"

In [5]:

DataFrame = pd.concat([Merch_df, Trans_df], axis=1).reindex(Merch_df.index)


# print(len(DataFrame.columns))
DataFrame



,TERMINAL_NO,MAININSTALLDATE,DISABLED_DATE,CATEGORY_NAME,CATEGORYCODE,PROVINCENAME,PROVINCECODE,CITYNAME,CITYCODE,PROJECTNAME,...,MOBILE,TERMINAL_NO,TRANDATE,RECENCYDATE,TRANNUMDAYS,FREQUENCY,AMOUNT,TRAN_PURCHASE,TRAN_CHARGE,TRAN_BALANCE


### Drop NaNs and not expected type of data in columns

In [6]:

Data = list(zip([0, 1, 2, " ", "ی", 5], [6, 7, 8, 9, 10, 11], ["A", "B", "C", "D", "E", "F"]))
df = pd.DataFrame(Data)
# print(df) 


# Drop NaN
DataFrame = DataFrame.dropna()




def returnExactType(stringOfType): # <class 'int'>  ->  int
    start = stringOfType.find("\'")+1
    stop = stringOfType.find(">")-1
    
    return stringOfType[start:stop]


def getMaximumTypeOfColumn(DataFrame, Column):
    types = set()
    
    for item in DataFrame[Column]:
        types.add(returnExactType(str(type(item))))

    typesCounter = dict.fromkeys(list(types), 0)
    
    for item in DataFrame[Column]:
        try : 
            typesCounter[returnExactType(str(type(item)))] += 1
            
        except:
            pass 
    
    res = max(typesCounter.items(), key=operator.itemgetter(1))[0] 
    
    
    if res == "int":
        return int
    
    elif res == "str":
        return str
    
    else:
        raise ValueError("type didn't defind 👎🏼")
    


def CheckItemsOfColumnByType(Type, DataFrame, Column) -> bool: # CheckItemsOfColumnByType
    return list(map(lambda x:type(x)==Type, list(DataFrame[Column])))
    

def Drop_FalseFlages(DataFrame, Column, Flages):
    
    df = DataFrame.copy()
    
    zipped = dict(zip( list(zip(df[Column], df.index)) , Flages))
        
    
    result = {key:val for key, val in zipped.items() if val == True}

    return list(result.keys()), len(DataFrame), len(result.keys())==len(DataFrame[Column])
    

# (dict_keys([(1, 0), (2, 2), (5, 5)]), 6)


def RemoveRowsOfFalseFlag(DataFrame, Column, Cleaned):
    
    if not Cleaned[2]:
        rangeOfFullIndex = set(range(Cleaned[1]))
        CleanedIndexed = set(map(lambda x:x[1], Cleaned[0]))
        
        notExpectedValues_Indexs =  rangeOfFullIndex.difference(CleanedIndexed)
        
        # print(notExpectedValues_Indexs)
        
        DataFrame = DataFrame.drop(notExpectedValues_Indexs, axis=0, inplace=True)
        
        # print("df : ", DataFrame)
        
        return DataFrame
    
    return DataFrame
        



ColumnTypes = []

for col in df:
    ColumnTypes.append(getMaximumTypeOfColumn(df, col))




index = 0
for col in df.columns:
    Flages = CheckItemsOfColumnByType(ColumnTypes[index], df, col)
    Cleaned = Drop_FalseFlages(df, col, Flages)
    print(Cleaned)
    DropedWrongs = RemoveRowsOfFalseFlag(df, col, Cleaned)
    index += 1
    
    

df.head()



([(0, 0), (1, 1), (2, 2), (5, 5)], 6, False)
([(6, 0), (7, 1), (8, 2), (11, 5)], 4, True)
([('A', 0), ('B', 1), ('C', 2), ('F', 5)], 4, True)


,0,1,2
0,0,6,A
1,1,7,B
2,2,8,C
5,5,11,F


### Seprate Date into two category 
    - two category with same len

In [7]:


def seprate_time(DataFrame,TimeStepMonth_Seperator=3 ,time_ColumnName="TRANDATE" ,index_ColumnName="TERMINAL_NO"):
    
    Time_df = DataFrame[time_ColumnName]
    
    
    # convert Jalalian to Gregorian
    df_jalaliToGregorian = []
    df_TerminalNumber = []
    
    # print(DataFrame)
    
    for index, row in DataFrame.iterrows():
        # print(int(row["TRANDATE"]))
        gregorian_date = JalaliDate(int(str(int(row["TRANDATE"]))[:4]), int(str(int(row["TRANDATE"]))[4:]),1).to_gregorian()
        df_TerminalNumber.append(int(row["TERMINAL_NO"]))
        df_jalaliToGregorian.append(gregorian_date)


    # Delete ME
    df_jalaliToGregorian.append(datetime.date(2000,10,1))
    df_jalaliToGregorian.append(datetime.date(2030,10,30))

    # Find Min and Max of Dates    
    date_min_max = pd.DataFrame(df_jalaliToGregorian).agg(["min", "max"])

    
    def time_plus_month(GivenDate, Months=TimeStepMonth_Seperator):
        date_format = '%Y/%m/%d'
        dtObj = datetime.datetime.strptime(GivenDate, date_format)
        future_date = dtObj + relativedelta(months=Months)
        return future_date

    middle = time_plus_month(str(date_min_max[0]['min']).replace("-","/"))
    
    Time_Category_Upper = []
    Time_Category_Under = [] 
    
    
    for date in df_jalaliToGregorian :
        if date > middle.date():
            Time_Category_Upper.append(date)
        else:
            Time_Category_Under.append(date)
    
    
    return Time_Category_Upper, Time_Category_Under
    
    # print(Time_Category_Upper) 
    # print(Time_Category_Under)
    
    
    
# print(seprate_time(Trans_df, 4))


### Make Unique Terminal number per sumOfFeatures For First Category of dates (1 to 3 month)
    - use maximum for RECENCYDATE

In [8]:
Trans_df.head()


,TERMINAL_NO,TRANDATE,RECENCYDATE,TRANNUMDAYS,FREQUENCY,AMOUNT,TRAN_PURCHASE,TRAN_CHARGE,TRAN_BALANCE


In [9]:
TerminalNo_Unique = list(Trans_df.TERMINAL_NO.unique())


# Columns = ["TERMINAL_NO"," RECENCYDATE", "TRANNUMDAYS", "FREQUENCY", "AMOUNT", "TC", "TRAN_PURCHASE", "TRAN_CHARGE", 'TRAN_BALANCE']
Columns = ["TERMINAL_NO"," RECENCYDATE", "TRANNUMDAYS", "FREQUENCY", "AMOUNT", "TRAN_PURCHASE", "TRAN_CHARGE", 'TRAN_BALANCE']


Max_RECENCYDATE = []
Sum_TRANNUMDAYS = []
Sum_FREQUENCY = []
Sum_AMOUNT = []
# Sum_TC = []
Sum_TRAN_PURCHASE = []
Sum_TRAN_CHARGE = []
Sum_TRAN_BALANCE = []

# Max_RECENCYDATE
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["RECENCYDATE"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Max_RECENCYDATE.append(max(Listed))

# print(Max_RECENCYDATE)

# Sum_TRANNUMDAYS
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["TRANNUMDAYS"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Sum_TRANNUMDAYS.append(sum(Listed))


# Sum_FREQUENCY
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["FREQUENCY"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Sum_FREQUENCY.append(sum(Listed))



# Sum_AMOUNT
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["AMOUNT"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Sum_AMOUNT.append(sum(Listed))



# # Sum_TC
# for Terminal in TerminalNo_Unique:
#     Listed = Trans_df["TC"][(Trans_df["TERMINAL_NO"]==Terminal)]
#     Sum_TC.append(sum(Listed))



# Sum_TRAN_PURCHASE
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["TRAN_PURCHASE"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Sum_TRAN_PURCHASE.append(sum(Listed))



# Sum_TRAN_CHARGE
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["TRAN_CHARGE"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Sum_TRAN_CHARGE.append(sum(Listed))


# Sum_TRAN_BALANCE
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["TRAN_BALANCE"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Sum_TRAN_BALANCE.append(sum(Listed))


# zipped = list(zip(TerminalNo_Unique, Max_RECENCYDATE, Sum_TRANNUMDAYS, Sum_FREQUENCY, Sum_AMOUNT, Sum_TC,
#                   Sum_TRAN_PURCHASE, Sum_TRAN_CHARGE, Sum_TRAN_BALANCE))

zipped = list(zip(TerminalNo_Unique, Max_RECENCYDATE, Sum_TRANNUMDAYS, Sum_FREQUENCY, Sum_AMOUNT,
                  Sum_TRAN_PURCHASE, Sum_TRAN_CHARGE, Sum_TRAN_BALANCE))




DataFrame = pd.DataFrame(zipped, columns=Columns)

DataFrame





,TERMINAL_NO,RECENCYDATE,TRANNUMDAYS,FREQUENCY,AMOUNT,TRAN_PURCHASE,TRAN_CHARGE,TRAN_BALANCE


### Cluster Terminals Based on (AMOUNT or TC) For Secend Category of dates (4 to 6 month)
    

In [10]:
TerminalNo_Unique = list(Trans_df.TERMINAL_NO.unique())

model = KMeans(n_clusters=3)
model = model.fit(scale(Sum_AMOUNT).reshape(-1,1))
# print(model.labels_)
idx = np.argsort(model.cluster_centers_.sum(axis=1))
lut = np.zeros_like(idx)
lut[idx] = np.arange(3)


Labels = {"CLUSTER" : np.array(lut[model.labels_])}
Labels_DataFrame = pd.DataFrame(Labels)
# print(Labels)



uniqueTerminal_Features_Merchant = pd.merge(DataFrame, Merch_df, on="TERMINAL_NO")


uniqueTerminal_Features_Merchant_Cluster = pd.concat([uniqueTerminal_Features_Merchant, pd.DataFrame(Labels)], axis=1)

DataFrame = uniqueTerminal_Features_Merchant_Cluster

DataFrame.head()


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required by the scale function.

In [ ]:
DataFrame["POSBRAND"].unique()

### Feature Engineering
    - Delete Column (Feature) that we don't need it
        - It's better to use Code instead name, eg : CityCode instead CityName
    - Make Better New Features From Old Features

    - is it True :
        - Use len Office address as attribute (Office on the right)
        - fillna with 0

In [ ]:

# MAININSTALLDATE
# Use Just year, eg : 1399/03/01 -> 1399 | NaN -> 0
DataFrame["MAININSTALLDATE"] = DataFrame["MAININSTALLDATE"].fillna(0)
DataFrame["MAININSTALLDATE"] = [int(str(item)[:4]) for item in DataFrame["MAININSTALLDATE"]]


# OFFICEADDRESS
# Use len insteal full address as a attribute
DataFrame["MAININSTALLDATE"] = DataFrame["MAININSTALLDATE"].fillna(0)
DataFrame["OFFICEADDRESS"] = [len(str(item)) for item in DataFrame["OFFICEADDRESS"]]


# Drop Columns
HaveToDrop = ["DISABLED_DATE", "CATEGORY_NAME", "PROVINCENAME",
                "CITYNAME", "PROJECTNAME", "AGENTNAME", "TEL1",
                "FIRSTNAMEFA", "LASTNAMEFA"]

DataFrame = DataFrame.drop(columns=HaveToDrop)


# MOBILE
# Find Operator and replace Mobile Number With Operator name, and then oneHot it!
def MobileOperator():
    DataFrame["MOBILE"] = DataFrame["MOBILE"].fillna(0)
    DataFrame["MOBILE"] = [int(str(item)[:3]) for item in DataFrame["MOBILE"]]

    mci = [912, 990, 919, 910, 911, 918, 917, 916, 915, 914, 913]
    irancell = [935, 936, 937, 938, 939, 901, 902, 903, 904, 905]

    index = 0
    for number in DataFrame["MOBILE"]:
        if number in mci:
            DataFrame["MOBILE"][index] = "mci"
            index += 1
        
        elif number in irancell:
            DataFrame["MOBILE"][index] = "irancell"
            index += 1
        
        else:
            DataFrame["MOBILE"][index] = "Another"
            index += 1
            
MobileOperator()
    

DataFrame.head()

### one hot encoding   
    - Use one hot endcoding for string categorized columns
        for example : column have two unique value men and women
        we use :
            10 for men
            01 for women
            00 for NaN 

In [ ]:

def oneHot_Column(DataFrame, ColumnName, inplace=False):
    uniqueValues = list(DataFrame[ColumnName].unique())
    uniqueValues_withOutNaN = []
        
    
    # uniqueValues without nan
    index = 0
    for uni in uniqueValues:
        try :
            math.isnan(uni)
            
            continue
            
        except:
            pass
        
        uniqueValues_withOutNaN.append(uni)
        
    
        
    Transformed = []
    
    # add full zero if nan value exist
    if len(uniqueValues) != len(uniqueValues_withOutNaN):
        Transformed.append("0"*len(uniqueValues_withOutNaN))
    
    
    
    index = 0
    for uniq in uniqueValues_withOutNaN:
        BasicZeros = [0] * len(uniqueValues_withOutNaN)
        BasicZeros[index] = 1
        
        Transformed.append("".join(map(str,BasicZeros)))
        index += 1
        
     
    unique_oneHot = dict(zip(uniqueValues, Transformed)) # dictionary of {each unique value : OneHot code} 


    # Replece in column    
    if inplace == True:
        for i in unique_oneHot.items():
            DataFrame[ColumnName].replace(i[0], int(i[1]), inplace=True)
            
    else:
        newDataFrame = DataFrame.copy()
        for i in unique_oneHot.items():
            newDataFrame[ColumnName] = newDataFrame[ColumnName].replace(i[0], int(i[1])) # Delete me when inplace is True
    
        return newDataFrame
    
            
    


In [ ]:
needToOneHot = ["TERMINALSTATUS", "POSTYPE","POSMODEL", "POSBRAND", "WORKFLOWCAPTION", "MOBILE"]

for column in needToOneHot:
    oneHot_Column(DataFrame, column, inplace=True)



DataFrame.head()


### Finally Check DataFrame
    - Check DataFrame not include any string and be clean 

In [ ]:
# Check Final Feed DataFrame has string or not

flag = False
errors = []

for column in DataFrame.columns:
    if any( [type(item) == str for item in DataFrame[column]]):
        flag = True
        errors.append(column,":" ,DataFrame[column][0],"|", DataFrame[column][1])

    
if flag:
    print(Fore.RED, "Final DataFrame includes String object 👎🏼, Strings Are : ")
    for err in errors:
        print(err)
        
    print(Fore.RESET)

else:
    print(Fore.GREEN, "Final DataFrame is clean of String object 👍🏼", Fore.RESET)




### Split Train Test

In [ ]:
# def Split_TrainTest(DataFrame, Train_ratio=0.7):
#     End_Train_index =  round(DataFrame.shape[0]*0.7)
#     Train = DataFrame.iloc[:End_Train_index]
#     Test =  DataFrame.iloc[End_Train_index:]
    
#     return Train, Test


# Train, Test = Split_TrainTest(DataFrame, 0.7)



# With Time Seprator

Train, Test = seprate_time(DataFrame)


print(f"{Fore.BLUE}Train : \n{Fore.RESET}Lenght : {len(Train)}")
print(Train.head())

print(f"\n{Fore.BLUE}Test : \n{Fore.RESET}Lenght : {len(Test)}")
print(Test.head())






TypeError: cannot unpack non-iterable NoneType object

### Fit Decision Tree

In [ ]:
Train_Features  = list(DataFrame.columns[ 1: (int(DataFrame.shape[1])-1) ]) # err

# Fit on Train
y = Train["CLUSTER"]
x = Train[Train_Features]

clf_Train = tree.DecisionTreeClassifier()
clf_Train.fit(x, y)

# Predict New Data Using RandomForest
clf_Train = RandomForestClassifier(n_estimators=10)

clf_Train = clf_Train.fit(x, y)

# values = [[10, 1, 4, 0, 0, 0]]
# prediction = clf_Train.predict(values)  # [1] OR [0]
# print( f"{values[0]}\n","Hired:", bool(prediction))



# # Fit on Test
# y = Test["CLUSTER"]
# x = Test[Train_Features]

# clf_Test = tree.DecisionTreeClassifier()
# clf_Test.fit(x, y)

# # Predict New Data Using RandomForest
# clf_Test = RandomForestClassifier(n_estimators=10)

# clf_Test = clf_Test.fit(x, y)



### Comparison

In [ ]:

Test_predicted = pd.DataFrame(dtype=int)

for row in Test:
    # values = [[10, 1, 4, 0, 0, 0]]
    values = [list(row)]
    prediction = clf_Train.predict(values)
    Test_predicted.append(prediction)


Test_StartIndex = round(DataFrame.shape[0]*0.7)


# result_df = pd.concat([DataFrame, Test_predicted], axis=1)


# print(result_df)


### Decesion Tree Graph 

In [ ]:
# dot_data = StringIO()
# tree.export_graphviz(clf, out_file=dot_data, feature_names=Train_Features)
# graph = pydot.graph_from_dot_data(dot_data.getvalue())

# plt = Image(pydot.create_png())
# display.svg(plt)